## Database setup and statistical analyis


In [10]:
# import libraries
# NOTE: global_vars should be edited to include local paths and credentials before use.
# If global_vars.py is created in the root dir remove the ignore/ prefix in the import statement below.
import ignore.global_vars as gv
import db_tools as dbt
import pandas as pd
import sqlalchemy
import sys
from IPython.display import display, HTML

In [7]:
# open connection and get table names
table_names = dbt.query("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
print(table_names)


Created SQLAlchemy engine for disaster_db
                table_name
0        geography_columns
1         geometry_columns
2          spatial_ref_sys
3       nri_shape_counties
4  nri_shape_census_tracts
5        NOAA_STORM_EVENTS
6        census_resilience


In [9]:
# Get table details
schema_info = dbt.query("""
    SELECT table_name, column_name, data_type 
    FROM information_schema.columns 
    WHERE table_schema='public'
    ORDER BY table_name, ordinal_position
""")
schema_info

Created SQLAlchemy engine for disaster_db


,table_name,column_name,data_type
0,NOAA_STORM_EVENTS,EPISODE_ID,bigint
1,NOAA_STORM_EVENTS,EVENT_ID,bigint
2,NOAA_STORM_EVENTS,EVENT_TYPE,text
3,NOAA_STORM_EVENTS,CZ_FIPS,text
4,NOAA_STORM_EVENTS,STATE_FIPS,text
...,...,...,...
976,spatial_ref_sys,srid,integer
977,spatial_ref_sys,auth_name,character varying
978,spatial_ref_sys,auth_srid,integer
979,spatial_ref_sys,srtext,character varying


In [11]:
# Create a pivot-like view
pivot_schema = schema_info.pivot_table(
    values='data_type', 
    index='column_name', 
    columns='table_name', 
    aggfunc='first', 
    fill_value=''
)

# Display as interactive table
display(pivot_schema)

table_name,NOAA_STORM_EVENTS,census_resilience,geography_columns,geometry_columns,nri_shape_census_tracts,nri_shape_counties,spatial_ref_sys
column_name,,,,,,,
AGRIVALUE,,,,,double precision,double precision,
ALR_NPCTL,,,,,double precision,double precision,
ALR_VALA,,,,,double precision,double precision,
ALR_VALB,,,,,double precision,double precision,
ALR_VALP,,,,,double precision,double precision,
...,...,...,...,...,...,...,...
geometry,,,,,USER-DEFINED,USER-DEFINED,
proj4text,,,,,,,character varying
srid,,,integer,integer,,,integer
